In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 


In [ ]:
df= pd.read_csv('/kaggle/input/titanic/Titanic.csv')

In [ ]:
df

In [ ]:
sns.catplot(data = df,y='Survived',x='Sex',col='Pclass',kind='bar')

На этом графике хорошо видно , что в независимости от класса билета, женщины спасались больше чем мужчины. А также пассажиры первого класса спасались больше.Стоит это уточнить.

In [ ]:
sns.catplot(data = df,hue = 'Survived', x = 'Pclass',kind='count')

Влияние выбора билета по классу правда имела значение. Пассажиры 3его класса заняли большинство погибших нежели 1го или 2го класса.

In [ ]:
df.Age.hist()

Можно заметить что чаще всего умирали в возрасте около 25-30 лет.

In [ ]:
df['Age_cat'] = pd.qcut(df.Age,7)
sns.catplot(data = df,hue = 'Survived', x = 'Age_cat',kind='count') 
plt.xticks(rotation=45)

График поделил возраста пассажиров на 7 групп, где можно увидеть что больше всего погибло людей в возрасте от21 до 26, а спаслось больше всего детей.

In [ ]:
child= df[df.Age<10]
sns.catplot(data = child,hue = 'Survived', x = 'Sex',kind='count')

Детей больше выжило, чем погибло.

In [ ]:
grand= df[df.Age>50]
sns.catplot(data = grand,hue = 'Survived', x = 'Sex',kind='count')

Среди возрастов за 50 мужчины умирали вероятнее, чем женщины.

Из этих графиков можно сделать вывод , что женщин и детей спасали впервую очередь.

In [ ]:
df = df.drop('Age_cat', axis=1)
df['family'] = df['Parch'] + df['SibSp']
sns.catplot(data = df,hue = 'Survived', x = 'family',
            kind='count')

На графике видно, что одинокие люди чаще всего погибали, а пассажиры с членами семьи от 1 до 3, чаще всего выживали.А пассажиры у кого было от 4 до 10 членов семьи больше погибали, чем выживали. Это может оказаться хорошим фактором для предсказания выживаемости.

In [ ]:
df['die'] = 1
df.loc[(df.Sex == 'male')&(df.Pclass == 3), 'die'] = 0
df.loc[(df.Sex == 'male')&(df.Age > 50), 'die'] = 0
df.loc[df['family']>3, 'die'] = 0

Добавляем колонку с предположением смерти, так как по статстике мужчины с 3им классом или мужчины с возрастом больше 50 лет погибали с наибольшей вероятностью.А также наличие больше 3 членов семьи влекло за собой смерть пассажира.

In [ ]:
df['alive'] = 0
df.loc[(df.Sex == 'female')&((df.Pclass == 1)&(df.Pclass == 2)), 'alive'] = 1
df.loc[df.Age < 6, 'alive'] = 1
df.loc[(df.Sex == 'female')&(df['family'] < 2), 'alive'] = 1
df.loc[(df.Sex == 'male')& (df['family'] < 2)&(df.Pclass == 1), 'alive'] = 1

Добавляем характеристику предсказания выживаемости.
По нашему анализу если пассажир женского пола,1го или 2го класса шанс выжить больше. Также если это ребенок и количество членов семьи <2. 
Если пассажир мужчина, то шанс на его выживаемость в случае если у него 1 класс, не более 2 членов семьи .

In [ ]:
categorical_cols= df.select_dtypes(include=['object']).columns
label_encoders={}

In [ ]:
from sklearn.preprocessing import LabelEncoder
for col in categorical_cols:
    le= LabelEncoder()
    df[col]=le.fit_transform(df[col])
    label_encoders[col]=le

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df['Age'].fillna(df['Age'].median(),inplace=True)

In [ ]:
df.Embarked.value_counts()

In [ ]:
df = df.drop(labels=['Cabin','Name','Ticket','Embarked'], axis=1)

Удаляем колонки с именем,портом посадки и номером кабины, т.к. эти значения не влияют на шанс выживаемости.
Порт не влияет, т.к. можно скинуть это на разницу в количестве пассарижов садившихся в одном порту.
Имя человека поможет нам определить пол человека, но так как в информации у каждого пассажира указан пол, больше информации от имени не требуется. Номер каюты мог помочь если бы мы знали расположение каждой каюты, но такой информацией мы не обладаем.

In [ ]:
df

In [ ]:
X=df.drop(columns=['Survived'])
y=df['Survived']

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

early_stopping=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

model=Sequential()
model.add(Dense(32,activation='relu',input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)))
model.add(Dense(16,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dense(8,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dense(4,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dense(2,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history=model.fit(X_train,y_train,epochs=20,batch_size=32,validation_split=0.2,verbose=0,callbacks=[early_stopping])

loss,accuracy=model.evaluate(X_test,y_test)
print(f'Test Accuracy:{accuracy:.2f}')

In [ ]:
loss=history.history.get('loss')
val_loss=history.history.get('val_loss')

plt.title("График потерь")
plt.plot(loss,label="Потери на тренировке")
plt.plot(val_loss,label="Потери на тесте")
plt.xlabel("Эпоха")
plt.ylabel("Потери")
plt.legend()
plt.show()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.svm import LinearSVC
from sklearn import datasets,linear_model

early_stopping=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

lasso = linear_model.Lasso()
cv_results = cross_validate(lasso, X, y, cv=3)

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

model=Sequential()
model.add(Dense(32,activation='relu',input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)))
model.add(Dense(16,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dense(8,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history=model.fit(X_train,y_train,epochs=20,batch_size=32,validation_split=0.2,verbose=0,callbacks=[early_stopping])

loss,accuracy=model.evaluate(X_test,y_test)
print(f'Test Accuracy:{accuracy:.2f}')

In [ ]:
loss=history.history.get('loss')
val_loss=history.history.get('val_loss')

plt.title("График потерь")
plt.plot(loss,label="Потери на тренировке")
plt.plot(val_loss,label="Потери на тесте")
plt.xlabel("Эпоха")
plt.ylabel("Потери")
plt.legend()
plt.show()

In [ ]:
from sklearn import datasets
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report

clf= SVC(kernel='poly',tol=1e-5,C=4,coef0=1,degree=5)
kf=KFold(n_splits=10)

for fold_number,(train_index,test_index) in enumerate(kf.split(X),start=1):
    X_train,X_test=X.iloc[train_index], X.iloc[test_index]
    y_train,y_test=y.iloc[train_index], y.iloc[test_index]

    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)

    cm=confusion_matrix(y_test,y_pred)
    cp=classification_report(y_test,y_pred)
    print(f"Номер обхода:{fold_number}")
    print(cm)
    print(cp)
    